In [1]:
pip install trtokenizer

In [2]:
!pip install TurkishStemmer
!pip install stanza
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-any-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 30.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
pip install zeyrek

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.0/931.0 kB 4.5 MB/s eta 0:00:00


In [14]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from snowballstemmer import stemmer
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import stanza
from TurkishStemmer import TurkishStemmer
import zeyrek
import spacy
nlp = spacy.load("tr_core_news_trf")

# Importing as module.
import tr_core_news_trf
nlp = tr_core_news_trf.load()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_excel('tweetData.xlsx')

In [6]:
df.shape

(15604, 2)

In [7]:
df = df.dropna()

In [8]:
df.shape

(1976, 2)

In [9]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [10]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [11]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [12]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [17]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

# Stop kelimeleri temizleme işlevi
def remove_stopwords_nltk(words):
    stop_words = set(stopwords.words('turkish'))
    return [word for word in words if word.lower() not in stop_words]

def spacy_lemme(text):

    doc = nlp(text)
    lemme = []
    for t in doc:

      lemme.append(t.lemma_)
    return ' '.join(lemme)

# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_nltk)
df['cleaned_text'] = df['cleaned_words'].apply(' '.join)
df['lemmatized_words'] = df['cleaned_text'].apply(spacy_lemme)
# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [18]:

# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['lemmatized_words']]

# FastText modelini eğit
fasttext_model = FastText(vector_size=100, window=3, min_count=1, epochs=10)
fasttext_model.build_vocab(corpus_iterable=tokenize_metni)
fasttext_model.train(corpus_iterable=tokenize_metni, total_examples=len(tokenize_metni), epochs=10)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [fasttext_model.wv[kelime] for kelime in kelimeler if kelime in fasttext_model.wv]
    if vektörler:
        return np.mean(vektörler, axis=0)
    else:
        return np.zeros(100)

df['fasttext_vektor'] = df['lemmatized_words'].apply(ortalama_vektor)
print(df['fasttext_vektor'] )



0        [-0.7792409, 0.043694146, -0.46043894, 0.89337...
1        [-0.69801444, 0.04008169, -0.41179046, 0.79877...
2        [-0.5540667, 0.031351972, -0.32667136, 0.63545...
3        [-0.8709221, 0.04920865, -0.51483727, 0.997837...
4        [-0.5448889, 0.03063547, -0.32236898, 0.625977...
5        [-0.681969, 0.038969334, -0.4018872, 0.7811186...
6        [-0.7228856, 0.041520137, -0.42723706, 0.82930...
7        [-0.82556665, 0.047420952, -0.48785836, 0.9458...
8        [-0.8261188, 0.045796115, -0.48894665, 0.94573...
9        [-0.93903804, 0.053066045, -0.55491185, 1.0765...
10       [-0.6401895, 0.03578892, -0.37878403, 0.734677...
11       [-0.6724921, 0.038043946, -0.3976432, 0.771033...
12       [-0.77099407, 0.04452308, -0.4563725, 0.885291...
13       [-0.70488805, 0.040746693, -0.41685507, 0.8089...
14       [-0.6419618, 0.036363672, -0.37874424, 0.73509...
15       [-0.7533818, 0.04218508, -0.44450587, 0.864198...
16       [-0.821912, 0.046803806, -0.48699886, 0.942896.